# Feb 6, 2025: construct graphs 
unit-sub, unit-ses

In [1]:
import os
import glob
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from itertools import product, combinations
from sklearn.covariance import GraphicalLasso
from scipy.stats import entropy, zscore
from sklearn.metrics import mutual_info_score
from joblib import Parallel, delayed
import graph_tool.all as gt 
import seaborn as sns

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

np.random.seed(args.SEED)

In [3]:
args.source = 'allen' #'allen'
args.space = 'ccfv2' #'ccfv2'
args.brain_div = 'whl' #'whl'
args.num_rois = 172 #216 #334 #162 #172
args.resolution = 200 #200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
os.system(f'mkdir -p {ROI_path}') 
TS_path = f'{ROI_path}/roi-timeseries'
os.system(f'mkdir -p {TS_path}')

0

In [5]:
def collect_timeseries(files):
    data_df = []

    pattern = re.compile(
        r"sub-(?P<sub>\w+)_ses-(?P<ses>\d+)_run-(?P<run>\d+)_task-(?P<task>\w+)_desc-ts\.txt"
    )
    # sub-SLC01_ses-1_run-11_task-rest_desc-ts.txt

    for file in tqdm(files):
        file_name = os.path.basename(file)
        match = pattern.match(file_name)
        metadata = match.groupdict()
        ts = np.loadtxt(file)
        metadata['ts'] = zscore(ts, axis=0, nan_policy='omit')
        data_df.append(metadata)
    data_df = pd.DataFrame(data_df).reset_index(drop=True)
    return data_df

In [6]:
data_df = collect_timeseries(
    files=sorted(glob.glob(f'{TS_path}/*', recursive=True))
)

  0%|          | 0/86 [00:00<?, ?it/s]

100%|██████████| 86/86 [00:00<00:00, 121.21it/s]


In [7]:
data_df

,sub,ses,run,task,ts
0,SLC01,1,11,rest,"[[-0.0955117651496492, -0.9480141361079346, 0...."
1,SLC01,1,15,rest,"[[0.2300138323488809, -1.7545658974959732, -1...."
2,SLC01,1,19,rest,"[[0.17467503871495776, 2.0563678304287225, 1.3..."
3,SLC01,2,10,rest,"[[2.193628914198822, -0.10906563819651435, 1.5..."
4,SLC01,2,6,rest,"[[0.6019518044701574, -0.13158998274825498, -1..."
...,...,...,...,...,...
81,SLC10,2,9,rest,"[[-1.4238413632568756, 1.0018252019653109, 1.1..."
82,SLC10,3,13,rest,"[[0.12182039381642772, 0.7583543245732659, 0.3..."
83,SLC10,3,17,rest,"[[-0.6009069370177516, -0.4645906291763468, -0..."
84,SLC10,3,5,rest,"[[-1.8217132895432822, 0.3352282395733613, 0.6..."


In [8]:
def get_cols(args):
    if args.DATA_UNIT == 'ses':
        cols = ['sub', 'ses', 'task']
    if args.DATA_UNIT == 'sub':
        cols = ['sub', 'task']
    return cols

In [9]:
# normalized mutual information
def optimal_bin_size(ts, method="fd"):
    """Computes the optimal number of bins for fMRI time series based on the selected method."""
    N = len(ts)  # Number of time points

    if method == "sturges":
        return int(np.ceil(np.log2(N) + 1))
    
    elif method == "rice":
        return int(np.ceil(2 * N ** (1/3)))

    elif method == "fd":  # Freedman-Diaconis Rule
        iqr = np.percentile(ts, 75) - np.percentile(ts, 25)
        bin_width = (2 * iqr) / (N ** (1/3))
        return int(np.ceil((np.max(ts) - np.min(ts)) / bin_width))

    elif method == "scott":  # Scott's Rule
        std_dev = np.std(ts)
        bin_width = (3.5 * std_dev) / (N ** (1/3))
        return int(np.ceil((np.max(ts) - np.min(ts)) / bin_width))
    
def compute_joint_density(ts1, ts2, bins=100):
    hist_xy, x_edges, y_edges = np.histogram2d(ts1, ts2, bins=bins, density=True)
    hist_x = np.histogram(ts1, bins=x_edges, density=True)[0]
    hist_y = np.histogram(ts2, bins=y_edges, density=True)[0]

    p_xy = hist_xy / np.sum(hist_xy) # joint density
    p_x = hist_x / np.sum(hist_x) # marginal of x
    p_y = hist_y / np.sum(hist_y) # marginal of y

    return p_xy, p_x, p_y

def compute_nmi(ts1, ts2, bins=100):
    # densities
    p_xy, p_x, p_y = compute_joint_density(ts1, ts2, bins)
    
    # entropies
    Hxy = entropy(p_xy.flatten(), base=2) # joint entropy: same as summing `- p_xy log(p_xy)` over each (x, y)
    Hx = entropy(p_x, base=2) 
    Hy = entropy(p_y, base=2)

    # mutual information
    Ixy = Hx + Hy - Hxy

    # normalize MI
    Ixy = Ixy / np.sqrt(Hx * Hy) if Hx > 0 and Hy > 0 else 0
    return Ixy

def compute_nmi_matrix(ts, bins=100, n_jobs=10):
    num_rois = ts.shape[1]
    nmi_matrix = np.zeros((num_rois, num_rois))

    def compute_nmi_pair(i, j):
        return compute_nmi(ts[:, i], ts[:, j], bins)
    
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_nmi_pair)(i, j)
        for i, j in combinations(range(num_rois), 2)
    )

    # fill nmi matrix
    for idx, (i, j) in enumerate(combinations(range(num_rois), 2)):
        nmi_matrix[i, j] = results[idx]
        nmi_matrix[j, i] = results[idx]
    
    return nmi_matrix

In [10]:
def compute_fc(args, ts):
    # ts.shape : time x rois
    if args.GRAPH_METHOD == 'pearson':
        fc = np.corrcoef(ts.T)
        fc -= np.diag(np.diag(fc))
    if args.GRAPH_METHOD == 'partial':
        model = GraphicalLasso(alpha=0.01)
        model.fit(ts)
        fc = -model.precision_ # inverse covariance matrix
    if args.GRAPH_METHOD == 'mutualinfo':
        bins = optimal_bin_size(ts)
        fc = compute_nmi_matrix(ts, bins=bins, n_jobs=10)
    return np.nan_to_num(fc)

def threshold_fc(args, fc_matrix):
    keep_ratio = args.EDGE_DENSITY / 100
    
    fc_thresh = np.zeros_like(fc_matrix)

    # Compute percentile threshold
    fc_values = fc_matrix[np.triu_indices_from(fc_matrix, k=1)]  # Extract upper triangle
    if args.THRESHOLD=='signed':
        fc_values = fc_values  # Consider values with their signs
    if args.THRESHOLD=='unsigned':
        fc_values = np.abs(fc_values) # Consider values without their signs
    percentile_thresh = np.percentile(fc_values, 100 * (1 - keep_ratio))

    # Apply percentile threshold
    mask = fc_matrix >= percentile_thresh

    # construct edges by their definition
    if args.EDGE_DEF == 'binary':
        fc_thresh = mask
    elif args.EDGE_DEF == 'weighted':
        fc_thresh = fc_matrix * mask

    return fc_thresh

def make_graph(fc):
    fc = np.tril(fc, k=-1)

    edges = np.where(fc)
    edge_list = list(zip(*[*edges, fc[edges]]))

    g = gt.Graph(
        edge_list,
        eprops=[('weight', 'double')],
        directed=False, 
    )
    
    return g

def save_graph(g, identity):
    file = '_'.join([identity] + [f'desc-graph.gt.gz'])
    file = f'{GRAPH_path}/{file}'
    g.save(file)
    return file

In [11]:
GRAPH_DEFS = [f'constructed']
GRAPH_METHODS = [f'pearson'] # [f'pearson', f'mutualinfo']
THRESHOLDINGS = [f'signed', f'unsigned']
EDGE_DEFS = [f'binary', f'weighted']
EDGE_DENSITIES = [10, 20, 30] #[10, 15, 20, 25]
LAYER_DEFS = [f'individual'] #, f'multilayer']
DATA_UNITS = [f'ses', f'sub']

In [12]:
ITERS = product(
    GRAPH_METHODS, 
    THRESHOLDINGS, 
    EDGE_DEFS, 
    EDGE_DENSITIES, 
    DATA_UNITS,
)

In [13]:
for (
    GRAPH_METHOD, 
    THRESHOLD, 
    EDGE_DEF, 
    EDGE_DENSITY,
    DATA_UNIT
) in ITERS:
    args.GRAPH_DEF = f'constructed'
    args.GRAPH_METHOD = GRAPH_METHOD
    args.THRESHOLD = THRESHOLD
    args.EDGE_DEF = EDGE_DEF
    args.EDGE_DENSITY = EDGE_DENSITY
    args.LAYER_DEF = f'individual'
    args.DATA_UNIT = DATA_UNIT

    
    ROI_RESULTS_path = (
        f'{ROI_path}'
        f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
        f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
        f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
    )
    GRAPH_path = f'{ROI_RESULTS_path}/graphs'
    os.system(f'mkdir -p {GRAPH_path}')

    cols = get_cols(args)

    for key, group in tqdm(data_df.groupby(by=cols)):
        identity = '_'.join([f'{c}-{k}' for c, k in zip(cols, key)])
        ts = np.concatenate(group['ts'].to_list(), axis=0)
        ts = np.nan_to_num(ts)
        fc = compute_fc(args, ts)
        fc = threshold_fc(args, fc)
        g = make_graph(fc)
        file = save_graph(g, identity)
    # break

100%|██████████| 9/9 [00:00<00:00, 60.12it/s]
